In [6]:
# import subprocess
# subprocess.run("ollama serve", shell=True)

In [7]:
# import ollama
# #Then everytime you want to chat
# response = ollama.chat(model='llama3.2', messages=[
#   {
#     'role': 'user',
#     'content': 'Why is the sky blue?',
#   },
# ])
# print(response['message']['content'])

In [1]:
import pandas as pd
import os
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAI, AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader

In [2]:
from dotenv import load_dotenv
load_dotenv() # read local .env file
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')

In [10]:
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    deployment= 'genai-gpt-35-turbo',
    # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
    azure_endpoint=OPENAI_API_BASE, #"https://<your-endpoint>.openai.azure.com/", If not provided, will read env variable AZURE_OPENAI_ENDPOINT
    api_key=OPENAI_API_KEY, # Can provide an API key directly. If missing read env variable AZURE_OPENAI_API_KEY
    openai_api_version=OPENAI_API_VERSION, # If not provided, will read env variable AZURE_OPENAI_API_VERSION
)

In [13]:
import time

In [14]:
# Function to process text data in chunks and add to the FAISS index
def process_in_chunks(text_data, chunk_size, embeddings, index_name):
    # Check if the index already exists
    if os.path.exists(f"{index_name}.faiss") and os.path.exists(f"{index_name}.pkl"):
        vectorstore = FAISS.load_local(index_name, embeddings)
        print("Loaded existing FAISS index.")
    else:
        vectorstore = None
    
    for i in range(0, len(text_data), chunk_size):
        chunk = text_data[i:i + chunk_size]
        try:
            print(f"Processing chunk {i // chunk_size + 1}")
            temp_vectorstore = FAISS.from_texts(chunk, embeddings)
            
            if vectorstore:
                vectorstore.merge_from(temp_vectorstore)
            else:
                vectorstore = temp_vectorstore
            
            # Save the updated FAISS index after processing each chunk
            vectorstore.save_local(index_name)
            print(f"Chunk {i // chunk_size + 1} processed and index saved.")
            
        except Exception as e:
            print(f"Error processing chunk {i}: {e}")
        
        time.sleep(1)  # Sleep to respect the rate limit
    
    return vectorstore



In [15]:
index_name = "faiss_index"
# Load documents from a directory
loader = DirectoryLoader("./input-data", glob="**/*.txt")

print("dir loaded loader")

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n","\n"," ",""],
    chunk_size=1500,
    chunk_overlap=300,
    add_start_index=True,
)

# # Split documents into chunks
texts = text_splitter.split_documents(documents)
text_data = [doc.page_content for doc in texts]
# print(len(documents))
# if os.path.exists(f"{index_name}.faiss") and os.path.exists(f"{index_name}.pkl"):
#     vectorstore = FAISS.load_local(index_name, embeddings)
#     print("Loaded existing FAISS index.")
# else:
#     # texts = df.to_string().split("\n")
#     vectorstore = FAISS.from_texts(text_data, embeddings)
#     vectorstore.save_local(index_name)
#     print("Created and saved new FAISS index.")

# Process the data in chunks (adjust chunk_size based on rate limits)
chunk_size = 100  # You can adjust this based on your rate limit
vectorstore = process_in_chunks(text_data, chunk_size, embeddings, index_name)

dir loaded loader
Processing chunk 1
Error processing chunk 0: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}
